In [1]:
import tensorflow as tf

In [2]:
( x_train, y_train ), ( x_test, y_test ) = tf.keras.datasets.mnist.load_data()

x_train = tf.cast( x_train / 255.0, tf.float32 )
x_test = tf.cast( x_test / 255.0, tf.float32 )

y_train = tf.keras.utils.to_categorical( y_train, 10 )
y_test = tf.keras.utils.to_categorical( y_test, 10 )
print( x_train.shape, y_train.shape )

(60000, 28, 28) (60000, 10)


In [3]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [4]:
class Model:
    def __init__( self, name ):
        self.name = name
        self.__build_net()
        
    def __build_net( self ):
        self.W1 = tf.Variable( tf.random.normal( [ 3, 3, 1, 32 ], stddev = 0.01 ) )
        self.W2 = tf.Variable( tf.random.normal( [ 3, 3, 32, 64 ], stddev = 0.01 ) )
        self.W3 = tf.Variable( tf.random.normal( [ 3, 3, 64, 128 ], stddev = 0.01 ) )
        xavier = tf.keras.initializers.GlorotUniform()
        self.W4 = tf.Variable( xavier( [ 128 * 4 * 4, 625 ] ) )
        self.b4 = tf.Variable( tf.random.normal( [ 625 ] ) )
        self.W5 = tf.Variable( xavier( [ 625, 10 ] ) )
        self.b5 = tf.Variable( tf.random.normal( [ 10 ] ) )
        
    def __layer1( self, X, rate ):
        l1 = tf.nn.conv2d( X, self.W1,
                          strides = [ 1, 1, 1, 1 ], padding = 'SAME' )
        l1 = tf.nn.relu( l1 )
        l1 = tf.nn.max_pool( l1, ksize = [ 1, 2, 2, 1 ],
                           strides = [ 1, 2, 2, 1 ], padding = 'SAME' )
        return ( tf.nn.dropout( l1, rate ) )
    
    def __layer2( self, X, rate ):
        l2 = tf.nn.conv2d( self.__layer1( X, rate ), self.W2,
                          strides = [ 1, 1, 1, 1 ], padding = 'SAME' )
        l2 = tf.nn.relu( l2 )
        l2 = tf.nn.max_pool( l2, ksize = [ 1, 2, 2, 1 ],
                           strides = [ 1, 2, 2, 1], padding = 'SAME' )
        return ( tf.nn.dropout( l2, rate ) )
    
    def __layer3( self, X, rate ):
        l3 = tf.nn.conv2d( self.__layer2( X, rate ), self.W3,
                         strides = [ 1, 1, 1, 1 ], padding = 'SAME' )
        l3 = tf.nn.relu( l3 )
        l3 = tf.nn.max_pool( l3, ksize = [ 1, 2, 2, 1 ],
                           strides = [ 1, 2, 2, 1 ], padding = 'SAME' )
        return ( tf.nn.dropout( l3, rate ) )
    
    def __l3_flat( self, X, rate ):
        return ( tf.reshape( self.__layer3( X, rate ), [ -1, 128 * 4 * 4 ] ) )
                
    def __layer4( self, X, rate ):
        l4 = tf.nn.relu( tf.matmul( self.__l3_flat( X, rate ), self.W4 ) + self.b4 )
        return ( tf.nn.dropout( l4, rate ) )
                
    def Logits( self, X, rate ):
        X_img = tf.reshape( X, [ -1, 28, 28, 1 ] )        
        return ( tf.matmul( self.__layer4( X_img, rate ), self.W5 ) + self.b5 )
    
    def Predict( self, x_test, rate = 0 ):
        return ( self.Logits( x_test, rate ) )    
    
    def Cost( self, X, Y, rate ):
        return ( tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(
            logits = self.Logits( X, rate ), labels = Y ) ) )
    
    def correct_prediction( self, X, Y, rate ):
        return ( tf.equal( tf.argmax( self.Logits( X, rate ), 1 ),
                         tf.argmax( Y, 1 ) ) )
    
    def get_accuracy( self, x_test, y_test, rate = 0 ):
        return ( tf.reduce_mean( tf.cast( self.correct_prediction( x_test, y_test, rate ), tf.float32 ) ) )
    
    def train( self, X, Y, rate = 0.3 ):
        loss = lambda: self.Cost( X, Y, rate )
        tf.keras.optimizers.Adam( learning_rate ).minimize( loss,
                                                          [ self.W1, self.W2, self.W3, self.W4, self.W5,
                                                           self.b4, self.b5 ] )
        
        return loss()

In [5]:
m1 = Model( 'm1' )

In [6]:
print( 'Learning Started!' )
for epoch in range( training_epochs ):
    avg_cost = 0
    total_batch = int( len( x_train ) / batch_size )
    
    start_batch, end_batch = 0, batch_size
    for i in range( total_batch ):
        batch_xs = x_train[ start_batch : end_batch ]
        batch_ys = y_train[ start_batch : end_batch ]
        
        c = m1.train( batch_xs, batch_ys)
        avg_cost += c / total_batch
        
        start_batch += batch_size
        end_batch += batch_size
        
    print( 'Epoch: {:04d}, cost: {:.9f}'.format( epoch + 1, avg_cost ) )

Learning Started!
Epoch: 0001, cost: 0.449852079
Epoch: 0002, cost: 0.100004002
Epoch: 0003, cost: 0.077956147
Epoch: 0004, cost: 0.072517671
Epoch: 0005, cost: 0.072264887
Epoch: 0006, cost: 0.066059336
Epoch: 0007, cost: 0.069227867
Epoch: 0008, cost: 0.066650793
Epoch: 0009, cost: 0.071735561
Epoch: 0010, cost: 0.073252872
Epoch: 0011, cost: 0.079574421
Epoch: 0012, cost: 0.075862400
Epoch: 0013, cost: 0.072776549
Epoch: 0014, cost: 0.082470112
Epoch: 0015, cost: 0.077692308


In [8]:
tf.print( 'Accuracy: ', m1.get_accuracy( x_test, y_test ) )

Accuracy:  0.9905
